In [2]:
import json
import nltk
import spacy 
from datetime import datetime
import tweepy
import re
import string
import unicodedata
from gensim import corpora
from nltk.tokenize.toktok import ToktokTokenizer

In [3]:
# Let's use the Twitter Stream API to get tweets in real time
# We save our tweets to a file called "cats.json"
# override tweepy.StreamListener to add logic to on_status and on_error 
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status._json)
        with open ("insomnia.json", "a+") as stream_f:
            json_text = json.dumps(status._json)
            stream_f.write(json_text)
            stream_f.write('\n')
            
    def on_error(self, status_code):
        print("Error detected!")
        print (status_code)
        return False

In [4]:
def clean_tweet(tweet): 
    processed_tweet = {}
    processed_tweet["id"] = tweet['id']
    processed_tweet["user"] = tweet['user']['screen_name']
    processed_tweet["created_at"] = datetime.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y')
    created_at = datetime.strptime(tweet["created_at"],'%a %b %d %H:%M:%S +0000 %Y')
    processed_tweet["lang"] = tweet['lang']
    
    if tweet['lang'] != "en":
        processed_tweet["is_en"] = False
    else: 
        processed_tweet["is_en"] = True
        
    if "full_text" in tweet: 
        processed_tweet["text"] = tweet['full_text']
    elif "extended_tweet" in tweet:
        processed_tweet["text"] = tweet["extended_tweet"]["full_text"]
    elif "text" in tweet: 
        processed_tweet["text"] = tweet['text']
        
    if 'retweeted_status' in tweet:
        rt = tweet['retweeted_status']
        processed_tweet["is_rt"] = True
        processed_tweet["rt_user"] = rt['user']['screen_name']
        processed_tweet["rt_id"] = rt['id']        
        if "full_text" in rt:
            processed_tweet["rt_text"] = rt['full_text']
        elif "extended_tweet" in tweet['retweeted_status']:
            processed_tweet["rt_text"] = tweet['retweeted_status']['extended_tweet']["full_text"]
        elif "text" in rt: 
            processed_tweet["rt_text"] = rt['text']
    else: 
        processed_tweet["is_rt"] = False
            
    return processed_tweet

In [5]:
info = {"consumer_key": "C85NCy1PkZiRnAwnS0Abt5FlF",
        "consumer_secret": "JS17fB3b4s4BmYZ74n4EgwRjG1w7kvChU2U1f1sC2zpXMxIDa9",
        "access_token": "41218838-OVyxdHIDYAyf1EBezFy9XZaT20qFyjfPw5H6MlnFM",
        "access_secret": "gDDDmbnMPeBI3SMrRheifA8QfmOZs7f24ab718C0BtsW2"}

In [6]:
auth = tweepy.OAuthHandler(info['consumer_key'], info['consumer_secret'])
auth.set_access_token(info['access_token'], info['access_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [7]:
#Collecting Twitter Data
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=['insomnia'])

{'created_at': 'Sat Apr 04 04:05:26 +0000 2020', 'id': 1246287739848458240, 'id_str': '1246287739848458240', 'text': 'RT @InayahLamis: Insomnia. https://t.co/0BN6ip2zsQ', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3112493373, 'id_str': '3112493373', 'name': 'PAPO DA DON', 'screen_name': 'Arr0gantt', 'location': '🏊🏾\u200d♀️', 'url': None, 'description': 'God 1st.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 616, 'friends_count': 627, 'listed_count': 5, 'favourites_count': 12362, 'statuses_count': 29119, 'created_at': 'Wed Mar 25 02:10:44 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_colo

KeyboardInterrupt: 

In [8]:
# load the raw tweets found from the search and stream apis into the same list for processing
tweet_data = []

with open("insomnia.json") as stream_f:
    for line in stream_f:
        json_line = json.loads(line)
        tweet_data.append(json_line)
        
# filter all of the raw tweets by turning them into clean_tweet objects
# the filtering is taken care of in the class function
filtered_data = []
for elem in tweet_data: 
    filtered_tweet = clean_tweet(elem)
    filtered_data.append(filtered_tweet)

In [9]:
filtered_data

[{'id': 1246239241534951424,
  'user': 'fixiontheboy',
  'created_at': datetime.datetime(2020, 4, 4, 0, 52, 43),
  'lang': 'en',
  'is_en': True,
  'text': 'Well shit den. Insomnia is a bellend. How do you guys do this',
  'is_rt': False},
 {'id': 1246239251286630401,
  'user': '420ogstunner',
  'created_at': datetime.datetime(2020, 4, 4, 0, 52, 46),
  'lang': 'en',
  'is_en': True,
  'text': 'RT @Medicat67671113: Blue Cheese is an indica Cross of Blueberry/Cheese strains from UK. The heavy indica effects will help you feel relaxe…',
  'is_rt': True,
  'rt_user': 'Medicat67671113',
  'rt_id': 1226156420552429576,
  'rt_text': 'Blue Cheese is an indica Cross of Blueberry/Cheese strains from UK. The heavy indica effects will help you feel relaxed at the end of the day//can provide relief for insomnia,stress, depression, bipolar, PTSD,It has a Sweet,blueberry flavor &amp; THC is  22%+. https://t.co/UJ6YV0T8oR https://t.co/uRW3y9ko8O'},
 {'id': 1246239263848611841,
  'user': 'avibecalledti

In [10]:
# create a list of all the tweet text 
# we filter out all tweets that are not English
tweet_text = []
for tweet in filtered_data:
    if tweet["is_en"]:
        if tweet["is_rt"]: 
            tweet_text.append(tweet["rt_text"].replace("\n", " "))
        else:
            tweet_text.append(tweet["text"].replace("\n", " "))

In [11]:
tweet_text

['Well shit den. Insomnia is a bellend. How do you guys do this',
 'Blue Cheese is an indica Cross of Blueberry/Cheese strains from UK. The heavy indica effects will help you feel relaxed at the end of the day//can provide relief for insomnia,stress, depression, bipolar, PTSD,It has a Sweet,blueberry flavor &amp; THC is  22%+. https://t.co/UJ6YV0T8oR https://t.co/uRW3y9ko8O',
 '@temsss__ insomnia?',
 'Insomnia. https://t.co/0BN6ip2zsQ',
 'Banana OG is an indica-dominant cross of OG Kush and Banana,THC level of 19%, making it ideal for long-time tokers and those new or sensitive to cannabis. It can treat chronic pain, depression,insomnia, and chronic fatigue. best for evening usage. https://t.co/UJ6YV0T8oR https://t.co/CQ4ZYRL3nF',
 'i didnt know what to do with my insomnia and anxiety jud last night😆 yawa kamatay',
 'Insomnia. https://t.co/0BN6ip2zsQ',
 'Insomnia. https://t.co/0BN6ip2zsQ',
 '@Madonna “Those who know can’t sleep.” Soon the entire world will have insomnia. #ThesePeopleAr

In [12]:
#PreProcessing Data
# remove HTML links, mentions, hashtags, and special characters

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ' ')    
    return text

def strip_mentions(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def strip_hashtags(text):
    entity_prefixes = ['#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)
        
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [13]:
stripped_tweet_text = []
for elem in tweet_text:
    elem = strip_links(elem)
    elem = strip_mentions(elem)
    elem = strip_hashtags(elem)
    elem = elem.replace('RT', '')
    elem = remove_special_characters(elem)
    stripped_tweet_text.append(elem)

In [14]:
stripped_tweet_text

['Well shit den Insomnia is a bellend How do you guys do this',
 'Blue Cheese is an indica Cross of Blueberry Cheese strains from UK The heavy indica effects will help you feel relaxed at the end of the day can provide relief for insomnia stress depression bipolar PTSD It has a Sweet blueberry flavor amp THC is 22',
 'insomnia',
 'Insomnia',
 'Banana OG is an indica dominant cross of OG Kush and Banana THC level of 19 making it ideal for long time tokers and those new or sensitive to cannabis It can treat chronic pain depression insomnia and chronic fatigue best for evening usage',
 'i didnt know what to do with my insomnia and anxiety jud last night yawa kamatay',
 'Insomnia',
 'Insomnia',
 'Those who know cant sleep  Soon the entire world will have insomnia ThesePeopleAreSick',
 'Those who know cant sleep  Soon the entire world will have insomnia ThesePeopleAreSick',
 'Insomnia',
 'Insomnia',
 'Insomnia',
 '2am thank u lord for the insomnia I am receiving',
 'Shiro I will name this s

In [15]:
from textblob import TextBlob

for elem in stripped_tweet_text:
    print (elem)
    elem_textblob = TextBlob(elem)
    print (elem_textblob.sentiment)
    print ("----")

Well shit den Insomnia is a bellend How do you guys do this
Sentiment(polarity=-0.2, subjectivity=0.8)
----
Blue Cheese is an indica Cross of Blueberry Cheese strains from UK The heavy indica effects will help you feel relaxed at the end of the day can provide relief for insomnia stress depression bipolar PTSD It has a Sweet blueberry flavor amp THC is 22
Sentiment(polarity=0.03749999999999999, subjectivity=0.3125)
----
insomnia
Sentiment(polarity=0.0, subjectivity=0.0)
----
Insomnia
Sentiment(polarity=0.0, subjectivity=0.0)
----
Banana OG is an indica dominant cross of OG Kush and Banana THC level of 19 making it ideal for long time tokers and those new or sensitive to cannabis It can treat chronic pain depression insomnia and chronic fatigue best for evening usage
Sentiment(polarity=0.3477272727272727, subjectivity=0.509090909090909)
----
i didnt know what to do with my insomnia and anxiety jud last night yawa kamatay
Sentiment(polarity=0.0, subjectivity=0.06666666666666667)
----
Ins

In [16]:
#find sentiment vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [18]:
snt = analyser.polarity_scores('This is an examle of a happy tweet')
print(snt)

{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'compound': 0.5719}


In [19]:
snt = analyser.polarity_scores('This is an examle of a happy tweet')
print(snt)

{'neg': 0.0, 'neu': 0.654, 'pos': 0.346, 'compound': 0.5719}
